In [1]:
import torch
import torch_directml
import glob
import numpy as np
from PIL import Image
import os
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import copy
from tqdm.auto import tqdm
import glob
import time
torch.manual_seed(240)

In [2]:
dml = torch_directml.device()
dml

device(type='privateuseone', index=0)

In [3]:
train_transform = transforms.Compose([
        transforms.Resize(size = (224, 224)),
        transforms.ToTensor(),
        ])


test_and_val_transform = transforms.Compose([
        transforms.Resize(size = (224, 224)),
        transforms.ToTensor(),
        ])

In [4]:
dataset_path = "../splitted"
train_dataset = dsets.ImageFolder(root=os.path.join(dataset_path, "train"), transform=train_transform)
test_dataset = dsets.ImageFolder(root=os.path.join(dataset_path, "test"), transform=test_and_val_transform)
val_dataset = dsets.ImageFolder(root=os.path.join(dataset_path, "val"), transform=test_and_val_transform)

In [5]:
batch = 16
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, drop_last=True, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch, drop_last=True, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch, drop_last=True, shuffle=False)

In [6]:
dataloaders = {"train" : train_loader,
               "val" : val_loader}

In [7]:
def create_model():    
    model = torchvision.models.resnet152(weights='ResNet152_Weights.DEFAULT')
    n_features = model.fc.in_features
    model.fc = nn.Linear(n_features, 2)

    return model.to(dml)

In [8]:
base_model = create_model()

In [9]:
num_classes = 2
num_epochs = 10
learning_rate = 1e-5
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(base_model.parameters(), lr=learning_rate)

In [10]:
validation_loss = []
training_loss = []

def train_model():
    since = time.time()
    val_acc_history = []
    
    best_model_wts = copy.deepcopy(base_model.state_dict())
    best_acc = 0.0
    
    progress_bar_train = tqdm(range(num_epochs * len(train_loader)))
    progress_bar_eval = tqdm(range(num_epochs * len(val_loader)))
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                base_model.train()  # Set model to training mode
            else:
                base_model.eval()   # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                # labels = torch.tensor([l.item() for l in label])
                inputs = inputs.to(dml)
                labels = labels.to(dml)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'): 
                    outputs = base_model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        progress_bar_train.update(1)
                    elif phase == 'val':
                        progress_bar_eval.update(1)
                        
                running_loss += loss.item() * inputs.size(0)
                preds = preds.cpu()
                labels = labels.data.cpu()
                running_corrects += (preds == labels).sum()
            
            print("Lenght: ", len(dataloaders[phase].dataset))
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = float(running_corrects) / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'train':
              training_loss.append(epoch_loss)
            elif phase == 'val':
              validation_loss.append(epoch_loss)
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(base_model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    base_model.load_state_dict(best_model_wts)
    return base_model, val_acc_history

In [11]:
best_model, validation_acc_hist = train_model()

  0%|          | 0/41250 [00:00<?, ?it/s]

  0%|          | 0/5890 [00:00<?, ?it/s]

Epoch 0/9
----------


In [ ]:
model_path = "../trained-models/mycnn.pt"
torch.save(best_model.state_dict(), model_path)